# Chapter 9  Apriori Assossiation rule algorithm for grocery shop 

**Market Basket Analysis with Apriori Algorithm using Python**



## 1. Understanding the Data and Explore

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os # use commandline  
from google.colab import files
import io
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['Groceries_dataset.csv']) )
 

Saving Groceries_dataset.csv to Groceries_dataset.csv


In [3]:
data.head()

,Member_number,Date,itemDescription
0,1249,01-01-2014,citrus fruit
1,1249,01-01-2014,coffee
2,1381,01-01-2014,curd
3,1381,01-01-2014,soda
4,1440,01-01-2014,other vegetables


In [4]:
data.shape

(38765, 3)

# Data Exploration
Let’s first have a look at the top 10 most selling products:

In [5]:
import plotly.express as px
print("Top 10 frequently sold products(Tabular Representation)")
x = data['itemDescription'].value_counts().sort_values(ascending=True)[:10] #เรียงจากมากไปหาน้อย 10 อันดับ
fig = px.bar(x= x.index, y= x.values)
fig.update_layout(title_text= "Top 10 frequently sold products (Graphical Representation)", xaxis_title= "Products", yaxis_title="Count")
fig.show()

Top 10 frequently sold products(Tabular Representation)


In [ ]:
data["Year"] = data['Date'].str.split("-").str[-1]
data["Month-Year"] = data['Date'].str.split("-").str[1] + "-" + data['Date'].str.split("-").str[-1]
fig1 = px.bar(data["Month-Year"].value_counts(ascending=False), 
              orientation= "v", 
              color = data["Month-Year"].value_counts(ascending=False),
               labels={'value':'Count', 'index':'Date','color':'Meter'})

fig1.update_layout(title_text="Exploring higher sales by the date")

fig1.show()

# Observations:
From the above visualizations we can observe that:

Milk is bought the most, followed by vegetables.
Most shopping takes place in August / September, while February / March is the least demanding.

## 2. Data Preparation¶

I will use the Apriori algorithm to perform an association analysis.

The apriori method of the mlxtend library accepts the dataset as a True-False dataframe. I will use the data conversion methods of the **mlxtend library ** again to convert the data. Therefore, I will convert the raw data set to the format that these methods will require.

In [ ]:
# Groupby  Member_number  and Date 
gdata = (data.groupby(["Member_number"])['itemDescription']
           .apply(', '.join)
           .reset_index())
gdata.columns.name = None
print(gdata)



      Member_number                                    itemDescription
0              1000  sausage, whole milk, semi-finished bread, yogu...
1              1001  sausage, whole milk, rolls/buns, whole milk, s...
2              1002  frozen vegetables, other vegetables, butter, w...
3              1003  sausage, rolls/buns, root vegetables, detergen...
4              1004  dish cleaner, cling film/bags, canned beer, fr...
...             ...                                                ...
3893           4996  salty snack, tropical fruit, dessert, bottled ...
3894           4997  whole milk, canned beer, tropical fruit, white...
3895           4998                                   rolls/buns, curd
3896           4999  semi-finished bread, newspapers, butter milk, ...
3897           5000  fruit/vegetable juice, onions, soda, root vege...

[3898 rows x 2 columns]


In [ ]:
gdata

,Member_number,itemDescription
0,1000,"sausage, whole milk, semi-finished bread, yogu..."
1,1001,"sausage, whole milk, rolls/buns, whole milk, s..."
2,1002,"frozen vegetables, other vegetables, butter, w..."
3,1003,"sausage, rolls/buns, root vegetables, detergen..."
4,1004,"dish cleaner, cling film/bags, canned beer, fr..."
...,...,...
3893,4996,"salty snack, tropical fruit, dessert, bottled ..."
3894,4997,"whole milk, canned beer, tropical fruit, white..."
3895,4998,"rolls/buns, curd"
3896,4999,"semi-finished bread, newspapers, butter milk, ..."


In [ ]:
# Step1: I converted the data into list format. I separated the objects in each line with ','.
ldata = list(gdata["itemDescription"].apply(lambda x : x.split(',')))
ldata

[['sausage',
  ' whole milk',
  ' semi-finished bread',
  ' yogurt',
  ' whole milk',
  ' pastry',
  ' salty snack',
  ' canned beer',
  ' misc. beverages',
  ' sausage',
  ' hygiene articles',
  ' soda',
  ' pickled vegetables'],
 ['sausage',
  ' whole milk',
  ' rolls/buns',
  ' whole milk',
  ' soda',
  ' frankfurter',
  ' curd',
  ' beef',
  ' white bread',
  ' frankfurter',
  ' soda',
  ' whipped/sour cream'],
 ['frozen vegetables',
  ' other vegetables',
  ' butter',
  ' whole milk',
  ' tropical fruit',
  ' sugar',
  ' butter milk',
  ' specialty chocolate'],
 ['sausage',
  ' rolls/buns',
  ' root vegetables',
  ' detergent',
  ' frozen meals',
  ' dental care',
  ' rolls/buns',
  ' rolls/buns'],
 ['dish cleaner',
  ' cling film/bags',
  ' canned beer',
  ' frozen fish',
  ' pip fruit',
  ' whole milk',
  ' tropical fruit',
  ' rolls/buns',
  ' red/blush wine',
  ' chocolate',
  ' other vegetables',
  ' shopping bags',
  ' other vegetables',
  ' hygiene articles',
  ' whole milk

In [ ]:
!pip install mlxtend

In [ ]:
# Step2: I will apply the method of converting the data of mlxtend library into True-False
# dataframe.
# First, I install the mlxtend library for those who do not have it installed.

from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_data = te.fit(ldata).transform(ldata)
df = pd.DataFrame(te_data,columns=te.columns_)
pd.set_option('display.max_columns', None)
df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen chicken,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,kitchen utensil,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,make up remover,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic products,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,processed cheese,prosecco,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback,Instant food products,UHT-milk,artif. sweetener,bags,baking powder,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,preservation products,processed cheese,prosecco,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,F

## 3. Implementing Apriori Algorithm

In the output of the Apriori algorithm, we get the frequencies of each combination in the whole data set. For example, in the output below, only the frequency (frequency) of "BISCUIT" in the whole dataset is 0.35, while the frequency (frequency) of "BISCUIT and BREAD" in the whole dataset is 0.20.

The apriori algorithm was given a **min_support value of 0.2.** Thus, product associations that are below 0.2 support value in combinations have been eliminated. If the verbose argument is 1, it will tell us how many combinations there are. In our example, 42 combinations were formed. In the last case, we have 16 combinations. Thus, our combination of 42-16 = 26 remained below the value of 0.2 support and was considered as an insignificant rate that we would not add to our comments.

In [ ]:
from mlxtend.frequent_patterns import apriori
freq_items = apriori(df,min_support=0.15,use_colnames = True) #ทดลองเปลี่ยนค่า min_support
freq_items

,support,itemsets
0,0.197537,( bottled water)
1,0.337352,( other vegetables)
2,0.162648,( pastry)
3,0.322986,( rolls/buns)
4,0.206003,( root vegetables)
5,0.168805,( sausage)
6,0.163930,( shopping bags)
7,0.287327,( soda)
8,0.196254,( tropical fruit)
9,0.412776,( whole milk)


## 4. Create strong association rules

I will apply the association analysis to the combination of mlxtend's association_rules method and the data set that we have support values. I will interpret my latest output according to the values of "support" and "confidence" and suggest a sample action idea.

Interpretation of Sample Association Analysis Output:

The probability of BISCUIT and BREAD being seen together is 20% since support = 0.20.
When BISCUIT is taken, the probability of getting BREAD is around 57% since confidence = 0.571429.

By giving "min_threshold = 0.3", it is ensured that the values with "confidence" value below 0.3 are not brought.

In [ ]:
from mlxtend.frequent_patterns import association_rules
df_res = association_rules(freq_items, metric = "confidence", min_threshold = 0.3) #ทดลองเปลี่ยนค่า confidence
df_res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,( whole milk),( other vegetables),0.412776,0.337352,0.160852,0.389683,1.155121,0.021601,1.085743
1,( other vegetables),( whole milk),0.337352,0.412776,0.160852,0.476806,1.155121,0.021601,1.122383
2,( whole milk),( rolls/buns),0.412776,0.322986,0.155721,0.377253,1.168016,0.022400,1.087141
3,( rolls/buns),( whole milk),0.322986,0.412776,0.155721,0.482129,1.168016,0.022400,1.133919


## 5. Preparation for Data Filtering

In this section, taking the lowest and highest confidence values, these values will be used in data filtering and the idea of action will be proposed.

Let's find the highest confidence value. The output shows that the highest confidence value is 0.80.

In [ ]:
conf_max = df_res['confidence'].max()
conf_max

0.48212867355043687

Let's find the lowest confidence value. The output shows that the lowest confidence value is 0.307.

In [ ]:
conf_min = df_res["confidence"].min()
conf_min

0.37725295214418897

## 6. Data Filtering

Data are displayed. It is ranked in ascending order according to "confidence" value.

In [ ]:
df_res.sort_values("confidence",ascending = True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,( whole milk),( rolls/buns),0.412776,0.322986,0.155721,0.377253,1.168016,0.022400,1.087141
0,( whole milk),( other vegetables),0.412776,0.337352,0.160852,0.389683,1.155121,0.021601,1.085743
1,( other vegetables),( whole milk),0.337352,0.412776,0.160852,0.476806,1.155121,0.021601,1.122383
3,( rolls/buns),( whole milk),0.322986,0.412776,0.155721,0.482129,1.168016,0.022400,1.133919
